## Librerías

In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import pipeline

C:\Users\cande\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('new_processed.csv')

In [3]:
def classify(text, model):
    """
    Clasifica el texto y devuelve el resultado.
    
    Parameters:
        text (str): El texto a clasificar.
        
    Returns:
        dict: Un diccionario que contiene las etiquetas y los puntajes de la clasificación.
    """
    classifier = pipeline("zero-shot-classification", 
                      model=model)
    hypothesis_template = "This text presents information in a {} manner."
    return classifier(text, ["objective", "subjective"], hypothesis_template = hypothesis_template, multi_label=True)


def save_result(result, results_list):
    """
    Guarda el resultado en una lista y actualiza el contador de filas procesadas.
    
    Parameters:
        result (dict): El resultado de la clasificación.
        results_list (list): Lista donde se guardan los resultados.
        processed_count (list): Una lista que contiene el conteo actual de filas procesadas.
        total_count (int): El número total de filas que se van a procesar.
    """
    results_list.append(result)

def update_processed_count(processed_count, total_count):
    """
    Actualiza el contador de filas procesadas e imprime el progreso.
    
    Parameters:
        processed_count (list): Una lista que contiene el conteo actual de filas procesadas.
        total_count (int): El número total de filas que se van a procesar.
    """
    processed_count[0] += 1
    print(f"Procesadas {processed_count[0]} de {total_count} filas")

def iter_rows(df, text_column, model):
    """
    Procesa un DataFrame aplicando la función classify_and_save a cada texto en la columna especificada.
    
    Parameters:
        df (DataFrame): El DataFrame que contiene los textos.
        text_column (str): El nombre de la columna que contiene los textos.
    """
    results_list = []
    processed_count = [0]
    total_count = len(df)
    
    for index, row in df.iterrows():
        model_output = classify(row[text_column], model)
        save_result(model_output, results_list)
        update_processed_count(processed_count, total_count)
    return results_list


def process_results(results_list):
    """
    Procesa una lista de resultados de clasificación aplicando softmax a los puntajes.
    
    Parameters:
        results_list (list): Lista de resultados de clasificación.
        
    Returns:
        DataFrame: DataFrame de Pandas con los puntajes softmax para cada etiqueta.
    """
    results_processed = []

    for result in results_list:
        labels = result['labels']
        scores = result['scores']
        scores_tensor = torch.tensor(scores)
        softmax_scores = torch.softmax(scores_tensor, dim=0).tolist()
        result_dict = dict(zip(labels, softmax_scores))
        results_processed.append(result_dict)
    results_df = pd.DataFrame(results_processed)
    results_df['sentiment_label'] = results_df[["objective", "subjective"]].apply(
        lambda x: ["objective", "subjective"][np.argmax(x)], axis=1)
    return results_df

def process_dataframe(df, text_column, model, column_suffix='_processed'):
    """
    Procesa un DataFrame aplicando la función classify_and_save a cada texto en la columna especificada,
    y luego procesa los resultados utilizando la función process_results.
    
    Parameters:
        df (DataFrame): El DataFrame que contiene los textos.
        text_column (str): El nombre de la columna que contiene los textos.
        
    Returns:
        DataFrame: El DataFrame original junto con el DataFrame procesado de los resultados.
    """
    results_list = iter_rows(df, text_column, model)
    results_df = process_results(results_list)
    new_columns = {col: f"{col}{column_suffix}" for col in results_df.columns}
    results_df = results_df.rename(columns=new_columns)
    return df.join(results_df)


In [4]:
model_names = ["MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", "MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
                "MoritzLaurer/roberta-large-zeroshot-v2.0", "facebook/bart-large-mnli"]
result_names = ["mdeberta", "deberta", "roberta", "bart"]

for model_name, result_names in zip(model_names, result_names):
    df = process_dataframe(df, 'texto_limpio', model=model_name, column_suffix = result_names)

Procesadas 1 de 600 filas
Procesadas 2 de 600 filas
Procesadas 3 de 600 filas
Procesadas 4 de 600 filas
Procesadas 5 de 600 filas
Procesadas 6 de 600 filas
Procesadas 7 de 600 filas
Procesadas 8 de 600 filas
Procesadas 9 de 600 filas
Procesadas 10 de 600 filas
Procesadas 11 de 600 filas
Procesadas 12 de 600 filas
Procesadas 13 de 600 filas
Procesadas 14 de 600 filas
Procesadas 15 de 600 filas
Procesadas 16 de 600 filas
Procesadas 17 de 600 filas
Procesadas 18 de 600 filas
Procesadas 19 de 600 filas
Procesadas 20 de 600 filas
Procesadas 21 de 600 filas
Procesadas 22 de 600 filas
Procesadas 23 de 600 filas
Procesadas 24 de 600 filas
Procesadas 25 de 600 filas
Procesadas 26 de 600 filas
Procesadas 27 de 600 filas
Procesadas 28 de 600 filas
Procesadas 29 de 600 filas
Procesadas 30 de 600 filas
Procesadas 31 de 600 filas
Procesadas 32 de 600 filas
Procesadas 33 de 600 filas
Procesadas 34 de 600 filas
Procesadas 35 de 600 filas
Procesadas 36 de 600 filas
Procesadas 37 de 600 filas
Procesadas

In [5]:
df.to_csv('news_pbj.csv')